# Top Manufacturers and Aircraft Types Over Time

This notebook calculates and exports pie charts of:

- Top manufacturers over all time
- Top aircraft types over all time

In [1]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run utils.ipynb

In [2]:
# Top "N" manufacturers and aircraft types to include
top_n_manufacturers = 10
top_n_models = 10

In [3]:
import matplotlib.pyplot as plt

def export_manufacturer_counts_chart(export_folder_path, data):
    # Create labels for the legend
    labels = data.index
    sizes = data.values
    total = sizes.sum()
    legend_labels = [f"{label} ({size / total:.1%})" for label, size in zip(labels, sizes)]

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(8, 8))

    # Plot pie chart without labels
    wedges, texts = ax.pie(sizes, startangle=140)

    # Add title
    ax.set_title(f"Top {top_n_manufacturers} Aircraft Manufacturers")

    # Add legend below the chart
    ax.legend(
        wedges,
        legend_labels,
        title="Manufacturers",
        loc="lower center",
        bbox_to_anchor=(0.5, -0.15),
        ncol=2,  # Change number of columns to fit nicely
        frameon=False
    )

    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"top-manufacturers-over-time", "png")

    # Close the plot
    plt.close(fig)

In [4]:
import matplotlib.pyplot as plt

def export_aircraft_model_counts_chart(export_folder_path, data):
    # Create labels for the legend
    labels = data.index
    sizes = data.values
    total = sizes.sum()
    legend_labels = [f"{label} ({size / total:.1%})" for label, size in zip(labels, sizes)]

    # Plot pie chart without labels
    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts = ax.pie(sizes, startangle=140)

    # Set the title
    ax.set_title(f"Top {top_n_models} Aircraft Models")

    # Create the legend
    ax.legend(
        wedges,
        legend_labels,
        title="Aircraft Types",
        loc="lower center",
        bbox_to_anchor=(0.5, -0.15),
        ncol=2,
        frameon=False
    )

    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"top-aircraft-models-over-time", "png")

    # Close the plot
    plt.close(fig)

In [ ]:
# Construct the query
query = construct_query("aircraft", "sightings.sql", {})

try:
    # Run the query to load the data - this raises a ValueError if there's no data
    df = query_data("aircraft", query)

    # Calculate the manufacturer and model counts
    manufacturer_counts = df["Manufacturer"].value_counts().nlargest(top_n_manufacturers)
    model_counts = df["Model"].value_counts().nlargest(top_n_models)

    # Export the data to excel
    export_folder_path = get_export_folder_path()
    export_to_spreadsheet(export_folder_path, "top-manufacturers-over-time.xlsx", {
        "Manufacturers": manufacturer_counts
    })
    export_to_spreadsheet(export_folder_path, "top-aircraft-models-over-time.xlsx", {
        "Aircraft Models": model_counts,
    })

    # Export the charts
    export_manufacturer_counts_chart(export_folder_path, manufacturer_counts)
    export_aircraft_model_counts_chart(export_folder_path, model_counts)

except ValueError:
    print(f"WARNING: No data found")